In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/bloom for questions generation/arc_dataset.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   questionID                7787 non-null   object 
 1   originalQuestionID        7787 non-null   object 
 2   totalPossiblePoint        7787 non-null   int64  
 3   AnswerKey                 7787 non-null   object 
 4   isMultipleChoiceQuestion  7787 non-null   int64  
 5   includesDiagram           7787 non-null   int64  
 6   examName                  7787 non-null   object 
 7   difficulty                7787 non-null   int64  
 8   year                      7787 non-null   object 
 9   question                  7787 non-null   object 
 10  subject                   0 non-null      float64
 11  category                  7787 non-null   object 
dtypes: float64(1), int64(4), object(7)
memory usage: 730.2+ KB


In [ ]:
def linguistic_feature(df, question):
    df["number_of_words"] = df[question].apply(lambda x: len(str(x).split()))
    df["number_of_sentences"] = len(question.split("."))
    df["avg_word_length"] = df[question].apply(lambda x: sum(len(word) for word in str(x).split()) / len(str(x).split()))
    return df

In [ ]:
linguistic_feature(df,'question')

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,difficulty,year,question,subject,category,number_of_words,number_of_sentences,avg_word_length
0,Mercury_7220990,7220990,1,B,1,0,Mercury,8,2015,Which factor will most likely cause a person t...,NaN,Train,40,1,5.000000
1,MCAS_2007_8_5189,5189,1,B,1,0,MCAS,8,2007,Lichens are symbiotic organisms made of green ...,NaN,Train,32,1,5.000000
2,Mercury_SC_401169,401169,1,D,1,0,Mercury,5,2015,When a switch is used in an electrical circuit...,NaN,Train,39,1,4.538462
3,MCAS_2004_8_27,27,1,A,1,0,MCAS,8,2004,Which of the following is an example of an ass...,NaN,Train,21,1,4.761905
4,NYSEDREGENTS_2006_8_10,10,1,3,1,0,NYSEDREGENTS,8,2006,"Rocks are classified as igneous, metamorphic, ...",NaN,Train,25,1,5.240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,Mercury_SC_415480,415480,1,A,1,0,Mercury,3,2015,Which change would most likely increase the nu...,NaN,Train,18,1,5.222222
7783,Mercury_7172795,7172795,1,C,1,0,Mercury,7,2015,The skin is the largest organ in the human bod...,NaN,Train,43,1,3.581395
7784,NCEOGA_2013_8_59,59,1,D,1,0,North Carolina READY End-of-Grade Assessment,8,2013,Which food provides the most energy for the bo...,NaN,Train,23,1,4.217391
7785,Mercury_7219643,7219643,1,D,1,0,Mercury,8,2015,Screech owls have two color variations-red and...,NaN,Train,41,1,4.902439


In [ ]:
def count_pos_tags(dataframe, question_column):
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')

    unique_pos_tags = set()

    def update_unique_pos_tags(text):
        tokens = word_tokenize(text)
        pos_tags = pos_tag(tokens)
        unique_pos_tags.update(tag for word, tag in pos_tags)

    dataframe[question_column].apply(update_unique_pos_tags)

    for pos_tag_count in unique_pos_tags:
        pos_tag_column = f'POS_{pos_tag_count}_Count'
        dataframe[pos_tag_column] = dataframe[question_column].apply(lambda x: sum(1 for _, tag in pos_tag(word_tokenize(x)) if tag == pos_tag_count))

    return dataframe

In [ ]:
count_pos_tags(df, 'question')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,difficulty,year,question,...,POS_DT_Count,POS_._Count,POS_VBN_Count,POS_FW_Count,POS_PRP_Count,POS_WRB_Count,POS_TO_Count,POS_UH_Count,POS_NNPS_Count,POS_RBR_Count
0,Mercury_7220990,7220990,1,B,1,0,Mercury,8,2015,Which factor will most likely cause a person t...,...,8,1,1,0,0,0,1,0,0,0
1,MCAS_2007_8_5189,5189,1,B,1,0,MCAS,8,2007,Lichens are symbiotic organisms made of green ...,...,4,2,1,0,0,0,1,0,0,0
2,Mercury_SC_401169,401169,1,D,1,0,Mercury,5,2015,When a switch is used in an electrical circuit...,...,8,4,1,0,0,1,2,0,0,0
3,MCAS_2004_8_27,27,1,A,1,0,MCAS,8,2004,Which of the following is an example of an ass...,...,4,1,0,0,0,0,0,0,0,0
4,NYSEDREGENTS_2006_8_10,10,1,3,1,0,NYSEDREGENTS,8,2006,"Rocks are classified as igneous, metamorphic, ...",...,1,0,1,0,2,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7782,Mercury_SC_415480,415480,1,A,1,0,Mercury,3,2015,Which change would most likely increase the nu...,...,2,1,0,0,0,0,0,0,0,0
7783,Mercury_7172795,7172795,1,C,1,0,Mercury,7,2015,The skin is the largest organ in the human bod...,...,7,6,3,0,4,1,1,0,0,0
7784,NCEOGA_2013_8_59,59,1,D,1,0,North Carolina READY End-of-Grade Assessment,8,2013,Which food provides the most energy for the bo...,...,4,1,0,0,0,0,0,0,0,0
7785,Mercury_7219643,7219643,1,D,1,0,Mercury,8,2015,Screech owls have two color variations-red and...,...,4,2,1,0,0,0,0,0,0,0


In [ ]:
X = df.drop(['questionID', 'originalQuestionID', 'totalPossiblePoint', 'AnswerKey', 'isMultipleChoiceQuestion', 'includesDiagram', 'examName', 'difficulty', 'year', 'question', 'subject', 'category'], axis=1)
X.to_csv("/content/drive/MyDrive/QDET/tags.csv")

import pandas as pd
# from sklearn.preprocessing import StandardScaler



# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(X)
# scaled_df = pd.DataFrame(scaled_data, columns=X.columns)
# print(scaled_df)

In [ ]:
X = pd.read_csv("/content/drive/MyDrive/QDET/tags.csv")


In [6]:
def calculate_weighted_average(dataframe, new_column):
    num_columns = len(dataframe.columns)
    equal_weights = [1 / num_columns] * num_columns
    weighted_average = np.average(dataframe.values, axis=1, weights=equal_weights)
    dataframe[new_column] = weighted_average
    return dataframe

# X = calculate_weighted_average(X, "Difficulty")
# print(X)

In [ ]:
new_X = X.drop(['Difficulty'], axis=1)
new_X = new_X.iloc[:,1:]
new_Y = X['Difficulty']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_X,new_Y, test_size=0.25)
print(X_train)
print(X_test)

      number_of_words  number_of_sentences  avg_word_length  POS_CC_Count  \
5644               15                    1         3.733333             0   
2282               28                    1         4.035714             0   
6479               20                    1         4.750000             0   
4854               37                    1         5.054054             0   
510                24                    1         4.125000             2   
...               ...                  ...              ...           ...   
4249               38                    1         4.552632             1   
3361               35                    1         5.371429             0   
6502               78                    1         5.025641             0   
5070               24                    1         4.750000             0   
6534               51                    1         4.823529             0   

      POS_)_Count  POS_POS_Count  POS_IN_Count  POS_NNP_Count  POS_$_Count 

In [ ]:
X_train.to_csv("/content/drive/MyDrive/QDET/tags_order.csv")

In [8]:
df = pd.read_csv("/content/drive/MyDrive/QDET/tags_order.csv")
df = calculate_weighted_average(df, "Difficulty")

print(df["Difficulty"].min())
print(df["Difficulty"].max())

1.2715298885511646
169.94996492868842


In [ ]:


regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = np.mean((y_pred - y_test) ** 2)
mae = np.mean(np.abs(y_pred - y_test))
r2 = regressor.score(X_test, y_test)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R^2 Score:", r2)

Mean Squared Error: 2243.2773139161245
Mean Absolute Error: 40.57177025976315
R^2 Score: 0.017625621761155008


In [ ]:
import joblib
path = "/content/drive/MyDrive/QDET/linguistic_regressor.pkl"
joblib.dump(regressor, path)

['/content/drive/MyDrive/QDET/linguistic_regressor.pkl']